## Datetime processing to select absolute dates (2020/09/12) when given 'Monday'. How to know when a specific day of the month is a Monday?

Earliest date is 2020/09/03, which is a Thursday.

In [1]:
import pandas as pd
from datetime import datetime

In [2]:
boulder_data = pd.read_csv("~/Desktop/coding/boulder_data/boulderdata.csv")

In [3]:
boulder_data.head()

,gym_name,current_time,occupancy,waiting,weather_temp,weather_status
0,regensburg,2020/10/29 17:00,68.0,0.0,7.42,Rain
1,muenchen-west,2020/10/29 17:00,27.0,0.0,9.09,Rain
2,frankfurt,2020/10/29 17:00,59.0,0.0,10.75,Rain
3,dortmund,2020/10/29 17:00,11.0,0.0,9.99,Rain
4,muenchen-ost,2020/10/29 17:00,100.0,14.0,9.09,Rain


In [4]:
boulder_data.describe()

,occupancy,waiting,weather_temp
count,1984.000000,1984.000000,1984.000000
mean,47.423387,0.570565,14.425565
std,25.733930,3.857531,5.519889
min,0.000000,0.000000,2.350000
25%,28.000000,0.000000,10.157500
50%,45.000000,0.000000,13.980000
75%,66.000000,0.000000,18.130000
max,100.000000,40.000000,31.640000


In [5]:
## I WANT THE OPPOSITE LOL
day_name = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday','Sunday']
date = '2020/09/03'
day = datetime.strptime(date, '%Y/%m/%d').weekday()
print(day_name[day])

Thursday


In [6]:
day_name = ['monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday','sunday']
date = '2020/09/03'
day = datetime.strptime(date, '%Y/%m/%d').weekday()
print(day)
print(day_name[day])

3
thursday


In [7]:
# boulder_data[boulder_data.current_time.day == 3]


In [40]:
# maybe select start & end dates first (list through the current_time.unique() - maybe use generator?)
# for the dates in the list of dates:
#    if date == right day number, append to a list of dates
# then filter the df for the dates in the list
# then aggregate data on those
# similar to:
"""
closed_times = ['23:', '00:', '01:', '02:', '03:', '04:', '05:', '06:']

for i in closed_times:
    ave_temp_data = boulder_data[~boulder_data.current_time.str.contains(i)]
"""

"\nclosed_times = ['23:', '00:', '01:', '02:', '03:', '04:', '05:', '06:']\n\nfor i in closed_times:\n    ave_temp_data = boulder_data[~boulder_data.current_time.str.contains(i)]\n"

In [10]:
for x in boulder_data.current_time.unique():
    print(x)
    day_name = ['monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday','sunday']
    date = x
    day = datetime.strptime(date, '%Y/%m/%d %H:%M').weekday()
    print(day)
    print(day_name[day])
    break

2020/10/29 17:00
3
thursday


In [12]:
def return_all_dates(unique_array):
    yield unique_array

In [16]:
list(return_all_dates(boulder_data.current_time.unique()))[0][0]

'2020/10/29 17:00'

In [31]:
def avg_data_day(chosen_day):
    day_name = ['monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday','sunday']
    for ind, day in enumerate(day_name):
        if chosen_day.lower() == day:
            day_num = ind
    
    list_of_data = []
    for x in boulder_data.current_time.unique():
        date = x
        day = datetime.strptime(date, '%Y/%m/%d %H:%M').weekday()
        if day == day_num:
            list_of_data.append(x)
    return list_of_data

In [32]:
avg_data_day('Monday')

['2020/10/26 20:00',
 '2020/10/26 17:00',
 '2020/10/26 16:00',
 '2020/10/26 15:00',
 '2020/10/19 20:00',
 '2020/10/19 19:30',
 '2020/10/19 17:00',
 '2020/10/19 15:00',
 '2020/10/19 07:00',
 '2020/10/12 22:00',
 '2020/10/12 20:00',
 '2020/10/12 17:00',
 '2020/10/12 16:00',
 '2020/10/12 15:00',
 '2020/10/12 14:00',
 '2020/10/12 07:00',
 '2020/10/05 20:00',
 '2020/10/05 17:00',
 '2020/10/05 16:00',
 '2020/10/05 15:00',
 '2020/10/05 14:00',
 '2020/10/05 07:00',
 '2020/09/28 20:00',
 '2020/09/28 18:00',
 '2020/09/28 17:00',
 '2020/09/28 16:00',
 '2020/09/28 14:00',
 '2020/09/28 07:00',
 '2020/09/21 22:15',
 '2020/09/21 21:45',
 '2020/09/21 20:00',
 '2020/09/21 19:30',
 '2020/09/21 17:15',
 '2020/09/21 17:00',
 '2020/09/21 15:00',
 '2020/09/21 14:45',
 '2020/09/21 14:00',
 '2020/09/21 13:45',
 '2020/09/21 13:15',
 '2020/09/21 13:00',
 '2020/09/21 11:45',
 '2020/09/21 11:30',
 '2020/09/14 21:00',
 '2020/09/14 19:30',
 '2020/09/14 19:00',
 '2020/09/14 17:00',
 '2020/09/14 16:00',
 '2020/09/14 